In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.w3schools.com/xml/default.asp"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'id': 'leftmenuinnerinner'}).findAll(href=True)
for slug in pages:
    urls.append('https://www.w3schools.com/xml/' + slug.get('href'))

mainHeadings = []
subHeadingsH2 = []
subHeadingsH3 = []
links = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
listItems = []
tableData = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    subContent = soup.find('div', {"id": "main"})  # Page Sub Content
    if subContent is None:
        continue

    mainHeading = subContent.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    link = subContent.findAll('a')  # Page Links
    pageLinks = []
    if link is not None:
        for lin in link:
            if lin.get('href') is not None and not lin.get('href').startswith('http'):
                pageLinks.append('https://www.w3schools.com/xml/' + lin.get('href'))
            else:
                pageLinks.append(lin.get('href'))

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append('https://www.w3schools.com/xml/' + img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text.replace('\n', '')
            if 10 <= len(slug) <= 79:
                pageFacts.append(slug)
            elif 80 <= len(slug) <= 119:
                pageDefinitions.append(slug)
            elif 120 <= len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('div', {"class": "w3-code"})  # Codes and Outputs
    pageCodes = []
    if codeContent is not None:
        for slug in codeContent:
            pageCodes.append(slug.text.replace('\xa0', ''))

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)


    mainHeadings.append(pageMainHeadings)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    links.append(pageLinks)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    codes.append(pageCodes)
    tableData.append(pageTableData)
    listItems.append(pageListItems)

In [4]:
data = {'main_headings': mainHeadings, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'links': links, 'image_urls': imageURLs, 'facts': facts, 'definitions': definitions,
        'explanations': explanations, 'codes': codes, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('W3Schools_XML.csv')

In [5]:
a=pd.read_csv("W3Schools_XML.csv")
a

,Unnamed: 0,main_headings,sub_headings_h2,sub_headings_h3,links,image_urls,facts,definitions,explanations,codes,table_data,list_items
0,0,['XML Tutorial'],"['Why Study XML?', 'What You Will Learn', 'Imp...","['XML Example 1', 'XML Example 2', 'Header']","['https://www.w3schools.com/xml//default.asp',...",[],"['XML stands for eXtensible Markup Language.',...",['It is important (for all types of software d...,[],"['\n<?xml version=""1.0"" encoding=""UTF-8""?>\n<n...",[],['What is XML?|||How does XML work?|||How can ...
1,1,['Introduction to XML'],"['What is XML?', 'XML Does Not DO Anything', '...","['Reminder', 'Old Version', 'Reminder', 'New V...","['https://www.w3schools.com/xml/default.asp', ...",[],"['Maybe it is a little hard to understand, but...",['XML is a software- and hardware-independent ...,['The tags in the example above (like <to> and...,"[""\n<note>\n\n<to>Tove</to>\n\n<from>Jani</fro...",[],['XML stands for eXtensible Markup Language|||...
2,2,['How Can XML be Used?'],"['XML Separates Data from Presentation', 'XML ...",['Books.xml'],['https://www.w3schools.com/xml/xml_whatis.asp...,[],['XML is used in many aspects of web developme...,"['Because of this, with XML, there is a full s...","['In many HTML applications, XML is used to st...","['\n <?xml version=""1.0"" encoding=""UTF-8""?><bo...",[''],['Stocks and Shares|||Financial transactions||...
3,3,['XML Tree'],"['An Example XML Document', 'XML Tree Structur...",['XML Tree Structure'],['https://www.w3schools.com/xml/xml_usedfor.as...,['https://www.w3schools.com/xml/nodetree.gif'],['The image above represents books in this XML...,['XML documents form a tree structure that sta...,[],"['\n<?xml version=""1.0"" encoding=""UTF-8""?><boo...",[],[]
4,4,['XML Syntax Rules'],"['XML Documents Must Have a Root Element', 'Th...",[],"['https://www.w3schools.com/xml/xml_tree.asp',...",[],['In this example <note> is the root element:'...,['The syntax rules of XML are very simple and ...,"['""Opening and closing tags"" are often referre...",['\n <root>\n <child>\n <subchild>.....<...,['&lt;|||<|||less than&&&&gt;|||>|||greater th...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
119,119,['XML DOM - The Comment Object'],"['Try it Yourself - Examples', 'The Comment ob...",[],['https://www.w3schools.com/xml/dom_cdatasecti...,[],"['createComment() - Create a comment node', 'T...",[],['The examples below use the XML file books.xm...,[],['Property|||Description&&&data|||Sets or retu...,[]
120,120,['The XMLHttpRequest Object'],"['Try it Yourself - Examples', 'The XMLHttpReq...",[],['https://www.w3schools.com/xml/dom_comment.as...,[],['The XMLHttpRequest object is the developers ...,['With the XMLHttpRequest object you can updat...,['Retrieve the content of an ASP fileHow a web...,[],['Method|||Description&&&abort()|||Cancels the...,['Update a web page without reloading the page...
121,121,['XML DOM Parser Errors'],['XML Parser Error'],['Example'],"['https://www.w3schools.com/xml/dom_http.asp',...",[],"['When trying to open an XML document, a parse...","['If the parser encounters an error, it may lo...",[],"['\n <html><body><p id=""demo""></p><script>var ...",[],[]
122,122,['XSLT Elements Reference'],['XSLT Elements'],[],['https://www.w3schools.com/xml/dom_parser.asp...,[],['The XSLT elements from the W3C Recommendatio...,"['The links in the ""Element"" column point to a...",[],[],"[""Element|||Description&&&apply-imports|||Appl...",[]
